In [48]:
import os

In [49]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [50]:
os.chdir("../")

In [51]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [52]:
project_path = "C:/Users/RICH-FILES/Desktop/ml/Loan-Amount-Prediction"

os.chdir(project_path)

In [53]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    fit_intercept: bool
    n_jobs: int
    target_column: float
    

In [54]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories

In [ ]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LinearRegression
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name = config.model_name,
            fit_intercept=params.fit_intercept,
            n_jobs = params.n_jobs,
            target_column=schema.name
        )
        
        return model_trainer_config

In [56]:

import os
from credit_risk import logger
from sklearn.linear_model import LinearRegression 
import joblib
import pandas as pd


In [57]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop(self.config.target_column, axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        
        reg = LinearRegression()
        reg.fit(train_x, train_y)
    
        joblib.dump(reg, os.path.join(self.config.root_dir, self.config.model_name))

In [58]:
#define pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e    

[2025-03-03 22:48:24,697: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-03 22:48:24,700: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-03 22:48:24,704: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-03 22:48:24,706: INFO: common: created directory at: artifacts/model_trainer]
